In [7]:
from flask import Flask, render_template, request, jsonify
from flask_cors import cross_origin
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq

In [14]:
app = Flask(__name__)


@app.route('/', methods = ['GET']) ## route to display the home page
@cross_origin()
def homePage():
    return render_template('index.html')


@app.route('/review', methods = ['POST', 'GET'])
@cross_origin()
def index():
    if request.method == 'POST':
        try:
            searchstring = request.form['content'].replace(" ", '')
            flipkart_url = 'https://www.flipkart.com/search?q=' + searchstring
            uClient = uReq(flipkart_url)
            fliplart_page = uClient.read()
            uClient.close()
            flipkart_html = bs(flipkart_page, 'html.parser')
            bigboxes = flipkart_html.findAll("div", {"class": "_1AtVbE col-12-12"})
            box = bigboxes[0]
            productLink = 'https://www.flipkart.com' + box.div.div.div.a['href']
            prodRes = requests.get(productLink)
            prodRes.encoding = 'utf-8'
            prod_html = bs(prodRes.test, 'html.parser')
            print(prod_html)
            commentboxes = prod_html.find_all("div", {'class': "_16PBlm"})
            
            filename = searchstring + '.csv'
            fw = open(filename, 'w')
            headers = 'Product, Customer Name, Rating, Heading, Comment /n'
            fw.write(headers)
            reviews = []
            
            for commentbox in commentboxes:
                try:
                    name = commentbox.div.div.find_all('p', {'class': '_2sc7ZR _2V5EHH'})[0].text
                except:
                    name = 'No Name'
                    
                try:
                    rating = commentbox.div.div.div.div.text
                except:
                    rating = "No Rating"
                    
                try:
                    commentHead = comment.div.div.div.p.text
                except:
                    commentHead = 'No Comment Heading'
                    
                try:
                    comtag = commentbox.div.div.find_all('div', {'class': ''})
                    custComment = comtag[0].div.text
                except Exception as e:
                    print('Exception while creating dictionary: ', e)
                    
                mydict = {'Product': searchstring, "Name": name, "Rating": rating, 'CommentHead': commentHead,
                         "Comment": custComment}
                reviews.append(mydict)
            return render_template('results.html', reviews = reviews[0:(len(reviews)-1)])
        except Exception as e:
            print('The Exception message is: ', e)
            return 'Something went Wrong!!!'
    else:
        return render_template('index.html')

    
if __name__ == "__main__":
    app.run(debug = True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\sunso\anaconda3\envs\reviewscraptest\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
